In [1]:
import sys
sys.path.append('../lib/')

import pandas as pd
import numpy as np
import geopandas as gpd

import math

import scipy.interpolate
from copy import deepcopy

In [2]:
zones = gpd.read_file('../dbs/sweden/zones/DeSO/DeSO_2018_v2.shp')
#zones.loc[:, 'deso_3'] = zones.loc[:, 'deso'].apply(lambda x: x[:2])
#zones_subset = zones.loc[zones['deso_3'] == '14', :]
#zones_subset_info = dict(zip(zones_subset['deso'], zones_subset['geometry']))
#zone_name = list(zones_subset['deso'])

zones_info = dict(zip(zones['deso'], zones['geometry']))
zone_name = list(zones['deso'])

zones = zones.rename(columns={"deso": "zone"})[['zone', 'geometry']]


trips = pd.read_csv("../dbs/sweden/survey/day_trips.csv")
trips = trips.loc[:, ["sub_id", 'trip_id', 'trip_main_id', 'distance_main',
                              'date', "origin_main_deso", "desti_main_deso", 'trip_weight']]
trips = trips.drop_duplicates(subset=["sub_id", 'trip_id', 'trip_main_id'])
trips["T"] = trips["date"].apply(lambda x: pd.to_datetime(x))
trips = trips.loc[~trips["T"].apply(lambda x: x.weekday()).isin([5, 6]), :]
trips.dropna(axis=0, how='any', inplace=True)

odms = trips.groupby(['origin_main_deso', 'desti_main_deso']).sum()['trip_weight']
odms = odms.reindex(pd.MultiIndex.from_product([zones.zone, zones.zone], names=['ozone', 'dzone']), fill_value=0)


In [3]:
ODM = dict()
for i in range(0, len(zone_name)):
    element = dict()
    for j in range(0, len(zone_name)):
        element[zone_name[j]] = odms.at[zone_name[i], zone_name[j]]
    ODM[zone_name[i]] = element


bigzone_name = []
bigCover = []
subCover = []
old_name = zone_name[0][0:4]

bigzone_name.append(old_name)
subCover.append(zone_name[0])

for i in range(1, len(zone_name)):
    new_name = zone_name[i][0:4]

    if new_name == old_name:
        # this two zones belong the same big zone
        subCover.append(zone_name[i])

    if new_name != old_name:
        # find a new big zone
        #store old results
        bigCover.append(deepcopy(subCover))
        subCover.clear()

        #store new resutls
        bigzone_name.append(new_name)
        subCover.append(zone_name[i])

    old_name = new_name

# handle the lastest case
bigCover.append(subCover)


big_within = dict(zip(bigzone_name, bigCover))

In [4]:
geometry = dict(zip(zone_name, zones.geometry))

In [5]:
df_data = pd.read_csv('../results/distance_ratio_data.csv')
df_simulation = pd.read_csv('../results/distance_ratio_simulation.csv')


sp_data_sweden = scipy.interpolate.interp1d(df_data.loc[df_data.country == 'sweden', ['distance']].values.reshape(-1),
                                            df_data.loc[df_data.country == 'sweden', ['ratio']].values.reshape(-1), bounds_error = False, fill_value = 1.5)

sp_data_simulation = scipy.interpolate.interp1d(df_simulation.loc[:, ['distance']].values.reshape(-1),
                                                df_simulation.loc[:, ['ratio']].values.reshape(-1), bounds_error = False, fill_value = 1.5)

In [6]:
ODM_truth = dict()
travelDemand_d = dict()
travelDemand_D = dict()
travelDemand_D_simulation = dict()
for i in range(0, len(bigzone_name)):
    element = dict()
    element1 = dict()
    element2 = dict()
    element3 = dict()
    for j in range(0, len(bigzone_name)):
        average_daily_trips = 0
        demand_d = 0
        demand_D = 0
        demand_D_simultion = 0
        if i == j:
            for begin in range(0, len(big_within[bigzone_name[i]])):
                for end in range(0, len(big_within[bigzone_name[i]])):
                    if begin != end:
                        average_daily_trips = average_daily_trips + ODM[big_within[bigzone_name[i]][begin]][big_within[bigzone_name[i]][end]]

                        deltax = (geometry[big_within[bigzone_name[i]][begin]].centroid.x - geometry[big_within[bigzone_name[j]][end]].centroid.x) / 1000                    
                        deltay = (geometry[big_within[bigzone_name[i]][begin]].centroid.y - geometry[big_within[bigzone_name[j]][end]].centroid.y) / 1000
                    
                        distance = math.sqrt(deltax * deltax + deltay * deltay)
                        demand_d = demand_d + ODM[big_within[bigzone_name[i]][begin]][big_within[bigzone_name[j]][end]] * distance
                        demand_D = demand_D + ODM[big_within[bigzone_name[i]][begin]][big_within[bigzone_name[j]][end]] * float(sp_data_sweden(distance)) * distance
                        demand_D_simultion = demand_D_simultion + ODM[big_within[bigzone_name[i]][begin]][big_within[bigzone_name[j]][end]] * float(sp_data_simulation(distance)) * distance
                    else:
                        average_daily_trips = average_daily_trips + ODM[big_within[bigzone_name[i]][begin]][big_within[bigzone_name[i]][end]]
        if i != j:
            for begin in range(0, len(big_within[bigzone_name[i]])):
                for end in range(0, len(big_within[bigzone_name[j]])):
                    average_daily_trips = average_daily_trips + ODM[big_within[bigzone_name[i]][begin]][big_within[bigzone_name[j]][end]]

                    deltax = (geometry[big_within[bigzone_name[i]][begin]].centroid.x - geometry[big_within[bigzone_name[j]][end]].centroid.x) / 1000                    
                    deltay = (geometry[big_within[bigzone_name[i]][begin]].centroid.y - geometry[big_within[bigzone_name[j]][end]].centroid.y) / 1000
                    
                    distance = math.sqrt(deltax * deltax + deltay * deltay)
                    demand_d = demand_d + ODM[big_within[bigzone_name[i]][begin]][big_within[bigzone_name[j]][end]] * distance
                    demand_D = demand_D + ODM[big_within[bigzone_name[i]][begin]][big_within[bigzone_name[j]][end]] * float(sp_data_sweden(distance)) * distance
                    demand_D_simultion = demand_D_simultion + ODM[big_within[bigzone_name[i]][begin]][big_within[bigzone_name[j]][end]] * float(sp_data_simulation(distance)) * distance
        
        element[bigzone_name[j]] = average_daily_trips
        element1[bigzone_name[j]] = demand_d
        element2[bigzone_name[j]] = demand_D
        element3[bigzone_name[j]] = demand_D_simultion

    ODM_truth[bigzone_name[i]] = element
    travelDemand_d[bigzone_name[i]] = element1
    travelDemand_D[bigzone_name[i]] = element2
    travelDemand_D_simulation[bigzone_name[i]] = element3



In [38]:
V_gt = pd.DataFrame(data=None, index=None, columns=['ozone_deso', 'dzone_deso', 'v_ij_gt'])

In [39]:
index = 0
for i in range(len(bigzone_name)):
    for j in range(i, len(bigzone_name)):
        ozone = "{}".format(bigzone_name[i])
        dzone = "{}".format(bigzone_name[j])
        V_gt.loc[index] = [ozone, dzone, ODM_truth[bigzone_name[i]][bigzone_name[j]]]
        index = index + 1

In [40]:
V_gt.head()

,ozone_deso,dzone_deso,v_ij_gt
0,0114,0114,5.144816e+07
1,0114,0115,2.166490e+05
2,0114,0117,5.804400e+05
3,0114,0120,0.000000e+00
4,0114,0123,5.046360e+05


In [42]:
V_gt.to_csv('../results/ODM_GT.csv', index=None)

In [43]:
Demand_d = pd.DataFrame(data=None, index=None, columns=['ozone_deso', 'dzone_deso', 'd_ij_gt'])


In [47]:
index = 0
for i in range(len(bigzone_name)):
    for j in range(i, len(bigzone_name)):
        ozone = "{}".format(bigzone_name[i])
        dzone = "{}".format(bigzone_name[j])
        
        Demand_d.loc[index] = [ozone, dzone, travelDemand_d[bigzone_name[i]][bigzone_name[j]]]
        #V_gt.append(row, ignore_index=True)
        index = index + 1


In [48]:
Demand_d.head()

,ozone_deso,dzone_deso,d_ij_gt
0,0114,0114,5.951112e+07
1,0114,0115,1.752398e+06
2,0114,0117,1.281179e+07
3,0114,0120,0.000000e+00
4,0114,0123,5.125630e+06


In [50]:
Demand_d.to_csv('../results/demand_d_GT.csv', index=None)

In [51]:
Demand_D = pd.DataFrame(data=None, index=None, columns=['ozone_deso', 'dzone_deso', 'D_ij_gt'])
index = 0
for i in range(len(bigzone_name)):
    for j in range(i, len(bigzone_name)):
        ozone = "{}".format(bigzone_name[i])
        dzone = "{}".format(bigzone_name[j])
        
        Demand_D.loc[index] = [ozone, dzone, travelDemand_D[bigzone_name[i]][bigzone_name[j]]]
        #V_gt.append(row, ignore_index=True)
        index = index + 1



In [52]:
Demand_D.head()

,ozone_deso,dzone_deso,D_ij_gt
0,0114,0114,1.002903e+08
1,0114,0115,2.545650e+06
2,0114,0117,1.714140e+07
3,0114,0120,0.000000e+00
4,0114,0123,7.344664e+06


In [53]:
Demand_D.to_csv('../results/demand_D_survey_GT.csv', index=None)

In [33]:
Demand_D_sim = pd.DataFrame(data=None, index=None, columns=['ozone_deso', 'dzone_deso', 'D_ij_sim_gt'])
index = 0
for i in range(len(bigzone_name)):
    for j in range(i, len(bigzone_name)):
        ozone = bigzone_name[i]
        dzone = bigzone_name[j]
        
        Demand_D_sim.loc[index] = [ozone, dzone,  travelDemand_D_simulation[bigzone_name[i]][bigzone_name[j]]]
        #V_gt.append(row, ignore_index=True)
        index = index + 1



In [34]:
Demand_D_sim.to_csv('../results/demand_D_sim_GT.csv', index=None)

In [35]:
Demand_D_sim.head()

,ozone_deso,dzone_deso,D_ij_sim_gt
0,0114,0114,8.961580e+07
1,0114,0115,2.245307e+06
2,0114,0117,1.560833e+07
3,0114,0120,0.000000e+00
4,0114,0123,6.464523e+06


In [36]:
ttt = pd.DataFrame(pd.read_csv('../results/demand_D_sim_GT.csv', converters={u'ozone_deso':str, u'dzone_deso':str, u'D_ij_sim_gt':float}))

In [37]:
ttt.head()

,ozone_deso,dzone_deso,D_ij_sim_gt
0,0114,0114,8.961580e+07
1,0114,0115,2.245307e+06
2,0114,0117,1.560833e+07
3,0114,0120,0.000000e+00
4,0114,0123,6.464523e+06
